In [1]:
from PIL import Image
import pandas as pd
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import seaborn as sns
from tqdm import tqdm
from urllib.parse import urlparse

In [2]:
def getAverageRGBN(image):
    im = np.array(image)
    w,h,d = im.shape
    im.shape = (w*h, d)
    return tuple(np.average(im, axis=0))

def myround(x, base=5):
    return base * round(x/base)

In [3]:
#list_img = os.listdir('/media/ruben/Data Drive/react-data/protest/amsterdam1/context_images')
list_img = os.listdir('D:/react-data/protest/amsterdam1/context_images')
list_img = [img for img in list_img if ".txt" not in img]
print(len(list_img))

30358


In [4]:
df = []
    
for img in tqdm(list_img):
    try:
        #im = Image.open('/media/ruben/Data Drive/react-data/protest/amsterdam1/context_images/{}'.format(img))
        im = Image.open('D:/react-data/protest/amsterdam1/context_images/{}'.format(img))
        width, height = im.size
        values_ = getAverageRGBN(im)
        values_ = "_".join([str(int(round(v))) for v in values_])
        df.append([width,height,values_,img])
    except Exception as e:
        pass

100%|████████████████████████████████████████████████████████████████████████████| 30358/30358 [16:29<00:00, 16.07it/s]


In [5]:
dfbu= df

In [6]:
df = pd.DataFrame(df,columns=['w','h','rgb','id'])
df.head()

,w,h,rgb,id
0,1000,667,137_133_133,00093698-62bb-11ea-ade3-80fa5b5388be.jpg
1,320,238,225_222_210,0076c3ac-62bb-11ea-ade3-80fa5b5388be.jpg
2,400,256,97_89_88,0076c96a-62bb-11ea-ade3-80fa5b5388be.jpg
3,400,400,133_133_133,00770e3e-62bb-11ea-ade3-80fa5b5388be.jpg
4,313,400,159_147_123,007718f2-62bb-11ea-ade3-80fa5b5388be.jpg


In [7]:
#df.columns = ['w','h','rgb','id']
df['w'] = df['w'].astype(int)
df['h'] = df['h'].astype(int)
df['aspratio'] = df['w'] / df['h'] 
df['aspratio'] = [round(i,1) for i in list(df['aspratio'])]

In [8]:
df['r'] = [myround(int(i.split("_")[0])) for i in list(df['rgb'])]
df['g'] = [myround(int(i.split("_")[1])) for i in list(df['rgb'])]
df['b'] = [myround(int(i.split("_")[2])) for i in list(df['rgb'])]
df['iid'] = df['aspratio'].astype(str) + df['r'].astype(str) + df['g'].astype(str) + df['b'].astype(str)

In [9]:
d_ = dict()

for c,unique in enumerate(list(set(list(df['iid'])))):
    ss = df[df['iid'] == unique]
    ss = list(ss['id'])
    d_.update({c:ss})

In [10]:
#list_txt = os.listdir('/media/ruben/Data Drive/react-data/protest/amsterdam1/context_images')
list_txt = os.listdir('D:/react-data/protest/amsterdam1/context_images')
list_txt = [img for img in list_txt if ".txt" in img]

In [11]:
url_img = dict()

for c,txt in enumerate(list_txt):
    #with open(os.path.join('/media/ruben/Data Drive/react-data/protest/amsterdam1/context_images',txt),'r',encoding='utf-8') as f:
    with open(os.path.join('D:/react-data/protest/amsterdam1/context_images',txt),'r',encoding='utf-8') as f:
        t = f.read()
        t = t.split(": ")[1]
        url_img.update({txt[:-4]:t})

In [24]:
vc = pd.DataFrame(df['iid'].value_counts())
vc['id'] = vc.index
vc.columns = ['count','iid']

In [25]:
list_larger = list(vc[vc['count'] > 5]['iid'])
#dfbu = df
df = df[df['iid'].isin(list_larger)]

In [31]:
networkdf = []

for image_fn in tqdm(list(set(df['id']))):
    own_url = url_img[image_fn[:-4]]
    connecting_fns = [i for i in list(df['id']) if own_url in url_img[i[:-4]]]
    
    for connfn in connecting_fns:
        source = image_fn[:-4]
        target = connfn[:-4]
        
        if source != target:
            networkdf.append([source,target])


  8%|█████▋                                                                      | 1093/14554 [00:07<01:27, 154.37it/s]


 17%|████████████▊                                                               | 2460/14554 [00:14<00:57, 210.46it/s]


 27%|████████████████████▎                                                       | 3901/14554 [00:21<00:51, 206.07it/s]


 37%|███████████████████████████▋                                                | 5314/14554 [00:29<00:48, 192.02it/s]


 46%|██████████████████████████████████▉                                         | 6701/14554 [00:37<00:46, 168.98it/s]


...

 65%|█████████████████████████████████████████████████                           | 9407/14554 [00:51<00:26, 191.08it/s]


 74%|███████████████████████████████████████████████████████▋                   | 10798/14554 [00:59<00:23, 157.48it/s]


 83%|██████████████████████████████████████████████████████████████▍            | 12105/14554 [01:06<00:14, 165.64it/s]


 92%|█████████████████████████████████████████████████████████████████████▎     | 13447/14554 [01:14<00:05, 188.73it/s]


100%|███████████████████████████████████████████████████████████████████████████| 14554/14554 [01:20<00:00, 181.49it/s]

In [33]:
ndf = pd.DataFrame(networkdf,columns=['s','t'])

In [34]:
ndf.to_csv('D:/react-data/protest/amsterdam1/st.csv',index=False)

In [40]:
nodes = [[i[:-4],i] for i in os.listdir('D:/react-data/protest/amsterdam1/context_images/') if ".txt" not in i]
nodes = pd.DataFrame(nodes,columns=['node','image'])

In [42]:
nodes.to_csv('D:/react-data/protest/amsterdam1/nodes.csv',index=False)